## Step2 with ErgoSCF

In [6]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
import util.libutil as comn
from libra_py import ERGO_methods
from libra_py import units
from libra_py.workflows.nbra import step2_ergoscf


Define the function to run the ErgosCF calculations for each geometry. This function essentially defines the core of the input file to the ErgoSCF code. Only the coordinates and the name of the executable may be changed.

Note the "Angstrom" keyword following the "molecule_inline" - it indicates that the input is provided in Angstrom units.

If your input (e.g. in the xyz trajectory file) is in the a.u. (Bohr) units, just remove the "Angstrom" keyword

In [7]:
def run(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline Angstrom
%sEOF
basis = "STO-3G"
use_simple_starting_guess=1
scf.create_mtx_files_F = 1
scf.create_mtx_file_S = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

Also define how to run the calculations for the "weird" geometry, when two snapshots are overlapped and form a "super-molecule". In this case, we don't really need to print out the Fock matrices

In [8]:
def run1(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline Angstrom
%sEOF
basis = "STO-3G"
use_simple_starting_guess=1
scf.create_mtx_file_S = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

Ok, so now we can run the step2 calculations! Just create the "res" dir, since this is where the results will be printed out

In [9]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo", "md_file":"md1.xyz",
          "isnap":0, "fsnap":5, "dt": 1.0 * units.fs2au
         }

os.system("mkdir res")
step2_ergoscf.run_step2(params, run, run1)

## Other examples (run for the debug purposes)

If we jneed to run an SCF calculations for a singe geometry

In [30]:
def run2(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline 
%sEOF
basis = "STO-3G"
scf.create_basis_func_coord_file = 1
scf.create_mtx_files_S_and_quit = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    
    print inp
    return inp

In [10]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo", "md_file":"md1.xyz"}
i = 0

E, MO = step2_ergoscf.do_step(i, params, run2)

NameError: name 'run2' is not defined

And we can print out the resulting matrices into a file

In [ ]:
E.show_matrix("E0.txt")
MO.show_matrix("MO.txt")

Here is another example - just to submit the ErgoSCF calculation for a given geometry and run type

In [ ]:
EXE = "/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo"
R = ERGO_methods.xyz_traj2gen_sp("md1.xyz", 0)
command = run2(EXE, R)
print command
os.system("%s" % (command))

We can read the produced matrices - in this case the Fock matrices. And we can specify which section of the matrix we want to read

In [ ]:
F = ERGO_methods.get_mtx_matrices("F_matrix_11.mtx")
F.show_matrix("F.txt")

F3 = ERGO_methods.get_mtx_matrices("F_matrix_11.mtx", [0,1,2], [0,1,2])
F3.show_matrix("Fa.txt")

Among a series of the Fock matrices, find the last one

In [ ]:
print ERGO_methods.find_last_file("F_matrix_", ".mtx")

Get a geometry for a "super-molecule" composed of two snapshots

In [13]:
i = 0
R = ERGO_methods.xyz_traj2gen_ovlp("md1.xyz", i, i+1)
print R

 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000
 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000



Here is an example to run the calculation of the overlap matrix between AOs in the two adjacent geometries

In [36]:
def do_ovlp_test(i, params, run):
    """

    Compute the overlap matrix in the AO basis for two geometries, i and i+1

    Args:
        i ( int ): index of the time step to be used from the trajectory file
        params ( dictionary ): the control parameters of the simulation
        
            * **params["EXE"]** ( string ): path to the ERGOSCF executable [ default: ergo ]
            * **params["md_file"]** ( string ): the name of the xyz file containing the trajectory - the 
                file should be in the general xyz format. [default: "md.xyz"]

        run (Python function ): the function that defines the ErgoSCF input generation - the user 
            has to define all the control parameters in it, to be able to run the ErgoSCF calculations

            Example:

                In the example below, the outermost quotes should be tripled 

                Note: the function should follw the signature shown here

                def run(EXE, COORDS):
                    inp = "#!bin/sh
                %s << EOINPUT > /dev/null
                spin_polarization = 0
                molecule_inline
                %sEOF
                basis = "STO-3G"
                use_simple_starting_guess=1
                scf.create_mtx_files_F = 1
                scf.create_mtx_file_S = 1
                XC.sparse_mode = 1
                run "LDA"
                EOINPUT
                " % (EXE, COORDS)
                    return inp

    Returns:
        CMATRIX(A, A): the matrix of the AO overlaps for two geometries, where A - is the size of the AO basis

    """

    # Now try to get parameters from the input
    critical_params = [ ] 
    default_params = { "EXE":"ergo", "md_file":"md.xyz"   }
    comn.check_input(params, default_params, critical_params)
    
    # Get the parameters
    EXE = params["EXE"]
    md_file = params["md_file"]
    
        
    # Make an input file for the overlap calculations 
    R = ERGO_methods.xyz_traj2gen_ovlp(md_file, i, i+1)
    
    # Run SCF calculations
    command = run(EXE, R)
    os.system("%s" % (command))
    
    # Get the overlap matrix
    S = ERGO_methods.get_mtx_matrices("S_matrix.mtx")
    
    # The basis is organized as:
    # atom 1 basis 1 (t+dt)
    # atom 1 basis 1 (t)
    # atom 1 basis m (t+dt)
    # atom 1 basis m (t)
    # ...
    # atom n basis 1 (t+dt)
    # atom n basis 1 (t)
    # atom n basis m (t+dt)
    # atom n basis m (t)
        
    norbs = S.num_of_cols/2
    act_sp1 = [1, 3,5,9,10,11]
    act_sp2 = [0, 2, 4, 6,7,8]
    
    """
    0.040000000     0.040000000     1.332000000     H (1)   1s   0
    0.020000000     0.030000000     1.332000000     H (0)   1s   1
    0.030000000     0.030000000     0.000000000    Li (1)   1s   2
    0.010000000     0.020000000     0.000000000    Li (0)   1s   3
    0.030000000     0.030000000     0.000000000    Li (1)   2s   4
    0.010000000     0.020000000     0.000000000    Li (0)   2s   5
    0.030000000     0.030000000     0.000000000    Li (1)   2p   6
    0.030000000     0.030000000     0.000000000    Li (1)   2p   7
    0.030000000     0.030000000     0.000000000    Li (1)   2p   8
    0.010000000     0.020000000     0.000000000    Li (0)   2p   9
    0.010000000     0.020000000     0.000000000    Li (0)   2p   10
    0.010000000     0.020000000     0.000000000    Li (0)   2p   11
    """
    
    #for i in xrange(norbs):
    #    act_sp1.append(2*i+1)
    #    act_sp2.append(2*i)
        
    S_sub = CMATRIX(norbs, norbs)
    pop_submatrix(S, S_sub, act_sp1, act_sp2)
    
    return S, S_sub    


In [38]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo", "md_file":"md1.xyz"}
i = 0

S, S_sub = do_ovlp_test(i, params, run2)
S.show_matrix("S.txt")
S_sub.show_matrix("S_sub.txt")

#!bin/sh
/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline 
 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000
 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000
EOF
basis = "STO-3G"
scf.create_basis_func_coord_file = 1
scf.create_mtx_files_S_and_quit = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT

